# The Fast Gradient Sign Method (FGSM)

* Taking an input image
* Making predictions on the image using a trained CNN
* Computing the loss of the prediction based on the true class label
* Calculating the gradients of the loss with respect to the input image
* Computing the sign of the gradient
* Using the signed gradient to construct the output adversarial image

### How does the Fast Gradient Sign Method work?
* Essentially, FGSM computes the gradients of a loss function (e.g., mean-squared error or categorical cross-entropy) with respect to the input image and then uses the sign of the gradients to create a new image (i.e., the adversarial image) that maximizes the loss
![alt text](pyimagesearch/fgsm_equation.png "Title")
where:

* adv_x: Our output adversarial image
* x: The original input image
* y: The ground-truth label of the input image
* epsilon: Small value we multiply the signed gradients by to ensure the perturbations are small enough that the human eye cannot detect them but large enough that they fool the neural network
* theta: Our neural network model
* J: The loss function

In [1]:
# import the necessary packages
from pyimagesearch.simplecnn import SimpleCNN
from pyimagesearch.fgsm import generate_image_adversary
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2

### load the MNIST dataset from disk

In [2]:
# load MNIST dataset and scale the pixel values to the range [0, 1]
print("[INFO] loading MNIST dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0
# add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
# one-hot encode our labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

[INFO] loading MNIST dataset...


We preprocess the MNIST dataset by:

* Scaling the pixel intensities from the range [0, 255] to [0, 1]
* Adding a batch dimension to the images
* One-hot encoding the labels

### initialize our SimpleCNN model

In [ ]:
# initialize our optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the simple CNN on MNIST
print("[INFO] training network...")
model.fit(trainX, trainY,
	validation_data=(testX, testY),
	batch_size=64,
	epochs=10,
	verbose=1)
# make predictions on the testing set for the model trained on
# non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print("[INFO] loss: {:.4f}, acc: {:.4f}".format(loss, acc))

[INFO] compiling model...
[INFO] training network...
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.1978 - accuracy: 0.9402 - val_loss: 0.0572 - val_accuracy: 0.9811
Epoch 2/10
60000/60000 [==============================] - 4s 64us/sample - loss: 0.0752 - accuracy: 0.9774 - val_loss: 0.0429 - val_accuracy: 0.9871
Epoch 3/10
60000/60000 [==============================] - 4s 64us/sample - loss: 0.0580 - accuracy: 0.9824 - val_loss: 0.0396 - val_accuracy: 0.9876
Epoch 4/10
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0464 - accuracy: 0.9858 - val_loss: 0.0413 - val_accuracy: 0.9866
Epoch 5/10
60000/60000 [==============================] - 4s 68us/sample - loss: 0.0398 - accuracy: 0.9873 - val_loss: 0.0361 - val_accuracy: 0.9879
Epoch 6/10
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0337 - accuracy: 0.9894 - val_loss: 0.0363 - val_accuracy: 0.9882
Epo

### generate some adversarial images using the FGSM now

In [ ]:
# loop over a sample of our testing images
for i in np.random.choice(np.arange(0, len(testX)), size=(10,)):
	# grab the current image and label
	image = testX[i]
	label = testY[i]
	# generate an image adversary for the current image and make
	# a prediction on the adversary
	adversary = generate_image_adversary(model,
		image.reshape(1, 28, 28, 1), label, eps=0.1)
	pred = model.predict(adversary)
	# scale both the original image and adversary to the range
	# [0, 255] and convert them to an unsigned 8-bit integers
	adversary = adversary.reshape((28, 28)) * 255
	adversary = np.clip(adversary, 0, 255).astype("uint8")
	image = image.reshape((28, 28)) * 255
	image = image.astype("uint8")
	# convert the image and adversarial image from grayscale to three
	# channel (so we can draw on them)
	image = np.dstack([image] * 3)
	adversary = np.dstack([adversary] * 3)
	# resize the images so we can better visualize them
	image = cv2.resize(image, (96, 96))
	adversary = cv2.resize(adversary, (96, 96))
	# determine the predicted label for both the original image and
	# adversarial image
	imagePred = label.argmax()
	adversaryPred = pred[0].argmax()
	color = (0, 255, 0)
	# if the image prediction does not match the adversarial
	# prediction then update the color
	if imagePred != adversaryPred:
		color = (0, 0, 255)
	# draw the predictions on the respective output images
	cv2.putText(image, str(imagePred), (2, 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.95, (0, 255, 0), 2)
	cv2.putText(adversary, str(adversaryPred), (2, 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.95, color, 2)
	# stack the two images horizontally and then show the original
	# image and adversarial image
	output = np.hstack([image, adversary])
	print("Real image is {} while it is predicted as {}".format(str(imagePred), str(adversaryPred)))
	#cv2.imshow("FGSM Adversarial Images", output)
	#cv2.waitKey(0)

Specifically, take note of the image.reshape call where we are ensuring 
the image has a shape of (1, 28, 28, 1). These values are:

* 1  : Batch dimension; we’re working with a single image here, so the value is trivially set to one.
* 28 : Height of the image
* 28 : Width of the image
* 1  : Number of channels in the image (MNIST images are grayscale, hence only one channel)

our preprocessing steps included scaling our training/testing images from the range [0, 255] to [0, 1] — to visualize our images with OpenCV, we now need to undo these preprocessing operations.

Initialize the color of our labels to be “green” (Line 76) if both the imagePred
and adversaryPred are equal. This will happen if our model can correctly label the adversarial image. Otherwise, we’ll update our prediction color to be red